In [13]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
from BeamSearchTree import BeamSearchTreeNode
import pyreader
import numpy as np
from attention import Batcher, construct_feed_dict, extract_results, get_evals,AttentionModel,get_initial_state

In [18]:
def find_path(tree, k=1):

    paths = []

    def search_tree(path, node, path_prob):
        if not node.children:
            paths.append((path, path_prob))
        else:
            for child in node.children:
                search_tree(path + [child.token_id], child, path_prob + np.log(child.probability))

    search_tree([], tree, 0)

    sorted_paths = sorted(paths, key=lambda x: x[1], reverse=True)
    return [path[0] for path in sorted_paths[:k]]

In [25]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}
def map_token(map, token):
    mask = 0
    if token.startswith("(*) "):
        mask = 1
        token = token.replace("(*) ", "")

    if token in map:
        return map[token], mask

    # Not in map, is it an identifier?
    if "|" in token:
        spl = token.split("|")
        if spl[1] in map:
            return map[spl[1]]
        elif spl[0] in map:
            return map[spl[0]]
        return pyreader.oov_id

    elif any([token.startswith(prefix) for prefix in [key for key in type_max_rands]]):
        return pyreader.oov_id

    raise KeyError(token)

In [3]:
seq_length = 1
batch_size = 1
hidden_size = 50
num_samples = 3
lambda_type='state'
max_attention=3
model_path = './model/2017-07-01--01-22--006942'
data_path='data_samples/mapping.map'
with open(data_path, "rb") as f:
    word_to_id = pickle.load(f)
vocab_size = len(word_to_id)
inv_map = {v: k for k, v in word_to_id.items()}

In [4]:
all_test_cases = ["import", "os", "\n", "class", "(*) Class291", ":", "\n", "§<indent>§", "def", "__init__", "(", "self", ",",
     "(*) arg123", ")", ":", "\n", "§<indent>§", "self", ".", "(*) attribute|attribute172", "=", "arg123", "\n", "\n",
     "§<dedent>§",
     "def", "(*) function234", "(", "self", ",", "(*) filename|arg432", ")", ":", "\n", "§<indent>§", "with", "open",
     "(", "filename|arg432", ",", "'r'", ")", "as", "(*) f|var76", ":", "\n", "§<indent>§", "(*) lines|var91", "=",
     "f|var76", ".",
     "readlines", "(", ")", "\n", "§<dedent>§", "return", "len", "(", "lines|var91", ")", "\n", "\n", "§<dedent>§",
     "def", "(*) func|function921", "(", "self", ",", "(*) arg191", ")", ":", "\n", "§<indent>§", "(*) var543", "=",
     "os", ".", "path", ".", "join", "(", "self", "."]

In [32]:
with tf.Graph().as_default(), tf.Session() as session:
    masks_ = tf.placeholder(tf.bool, [seq_length, batch_size, 1], name="masks")
    input_data_ = tf.placeholder(tf.int32, [seq_length, batch_size], name="inputs")
    targets_ = tf.placeholder(tf.float32, [seq_length, batch_size], name="targets")
    
    a = AttentionModel(input_data=input_data_,
                                 targets=targets_,
                                 masks=masks_,
                                 is_training=False,
                                 attention_num= 1,
                                 batch_size=batch_size,
                                 hidden_size=hidden_size,
                                 num_samples=num_samples,
                                 seq_length=seq_length,
                                 vocab_size=vocab_size,
                                 lambda_type=lambda_type,
                                 max_attention=max_attention)
    
    
    variables = tf.trainable_variables()
    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver(variables)
    saver.restore(session, os.path.join(model_path, "model.tf"))
    
    prediction_op = tf.nn.top_k(a.predict, 5)
    
    to_eval = [prediction_op[0], prediction_op[1]]
    evaluation = get_evals(to_eval, a)
    
    
    def run_network(token_id, state, att_states, att_ids, att_counts):
        att_mask = np.ones([1, 1])
        if isinstance(token_id, tuple):
            data = (np.array([[token_id[0]]]), np.array([[1]]), np.array([att_mask]), np.array([1]), np.array([1]))
        else:
            data = (np.array([[token_id]]), np.array([[1]]), np.array([att_mask]), np.array([1]), np.array([1]))
        feed_dict, _ = construct_feed_dict(a, data, state, att_states, att_ids, att_counts)

        results = session.run(evaluation, feed_dict)
        results, state, att_states, att_ids, alpha_states, att_counts, _ = extract_results(results, evaluation, 2,
                                                                                           a)
        return results, state, att_states, att_counts

    def beam(tree_node):
        # Populate the children of tree_node
        init_state, init_att_states, init_att_counts = tree_node.state
        results, state, att_states, att_counts = run_network(tree_node.token_id, init_state, init_att_states,
                                                             att_ids, init_att_counts)
        probs = results[0]
        predict_ids = results[1]
        for i in range(1,5):
            tree_node.add_child(BeamSearchTreeNode(predict_ids[0, i],np.ones([1, 1]),
                                                   (state, att_states, att_counts), probs[0, i]))

    def beam_search_recursive(tree, current_depth):
        if current_depth < 5:
            for child in tree.children:
                beam(child)
                beam_search_recursive(child, current_depth + 1)
    
    
    
    state, att_states, att_ids, att_counts = get_initial_state(a, session)
    count=0
    accuracy=0
    for i, token in enumerate(all_test_cases[:-5]):
        lst =np.ones([1, 1])
        token_id = map_token(word_to_id, token)
        
        if isinstance(token_id, tuple):
            data = (np.array([[token_id[0]]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
        else:
            data = (np.array([[token_id]]), np.array([[1]]), np.array([lst]), np.array([1]), np.array([1]))
        results, state, att_states, att_counts = run_network(map_token(word_to_id, token), state, att_states,
                                                                     att_ids, att_counts)
        root = BeamSearchTreeNode(map_token(word_to_id, all_test_cases[-1]),np.ones([1, 1]),
                                          (state, att_states, att_counts), 1)
        beam(root)
        beam_search_recursive(root, 1)
        path = find_path(root)[0]  # The most likely path
        actual = [map_token(word_to_id, t) for t in all_test_cases[i + 1:i + 5 + 1]]

        pred = " ".join([inv_map[t].replace("\n", "<newline>") for t in path])
        
        act = []
        for t in actual:
            try:
                act.append(inv_map[t[0]].replace("\n", "<newline>"))
            except:
                act.append(inv_map[t].replace("\n", "<newline>"))
        act = ' '.join (act)
        
        print('-'*40)
        print("Token(input): ", token.replace("\n", "<newline>"))
        print("Predicted: ", pred)
        print('Actual   : ', act)
        count+=count+1
        if pred == act:
            accurate += 1
    print("Accuracy: %f" % (accuracy / count))
#         results = session.run(evaluation, feed_dict)
#         results_match = ''.join(str(results[1][0][0]))
#         actual = [map_token(word_to_id, t) for t in all_test_cases[i + 1:i + 5 + 1]]
#         act = " ".join([inv_map[t[0]].replace("\n", "<newline>") for t in actual])
#         print('=============')
#         print('predict', inv_map[results[1][0][1]])
#         print('real', act)

Constructing Attention Cell
INFO:tensorflow:Restoring parameters from ./model/2017-07-01--01-22--006942\model.tf
----------------------------------------
Token(input):  import
Predicted:  . §PAD§ . . §PAD§
Actual   :  os <newline> class Class291 :
----------------------------------------
Token(input):  os
Predicted:  §PAD§ <newline> <newline> <newline> <newline>
Actual   :  <newline> class Class291 : <newline>
----------------------------------------
Token(input):  <newline>
Predicted:  . . §PAD§ = =
Actual   :  class Class291 : <newline> §<indent>§
----------------------------------------
Token(input):  class
Predicted:  §PAD§ <newline> <newline> <newline> <newline>
Actual   :  Class291 : <newline> §<indent>§ def
----------------------------------------
Token(input):  (*) Class291
Predicted:  . . §PAD§ = §OOV§
Actual   :  : <newline> §<indent>§ def __init__
----------------------------------------
Token(input):  :
Predicted:  §OOV§ <newline> <newline> <newline> <newline>
Actual   :  <

----------------------------------------
Token(input):  §<dedent>§
Predicted:  self = = §PAD§ ,
Actual   :  return len ( §OOV§ )
----------------------------------------
Token(input):  return
Predicted:  self ) <newline> <newline> <newline>
Actual   :  len ( §OOV§ ) <newline>
----------------------------------------
Token(input):  len
Predicted:  . §OOV§ ) <newline> §PAD§
Actual   :  ( §OOV§ ) <newline> <newline>
----------------------------------------
Token(input):  (
Predicted:  §OOV§ §PAD§ <newline> <newline> <newline>
Actual   :  §OOV§ ) <newline> <newline> §<dedent>§
----------------------------------------
Token(input):  lines|var91
Predicted:  §OOV§ ) <newline> self self
Actual   :  ) <newline> <newline> §<dedent>§ def
----------------------------------------
Token(input):  )
Predicted:  self ) <newline> §PAD§ <newline>
Actual   :  <newline> <newline> §<dedent>§ def function921
----------------------------------------
Token(input):  <newline>
Predicted:  §PAD§ <newline> <newlin

### 어큐러시가 낮은이유 : 
### 적은데이터셋트로 훈련을했었고, 훈련이 더필요함.